**PLEASE UPVOTE https://www.kaggle.com/code/abdulravoofshaik/top-3-solution-lgbm-mean**

In [ ]:
import numpy as np
import pandas as pd
from tqdm import tqdm

pd.set_option('display.max_columns', None)

In [ ]:
data = pd.read_csv('../input/tabular-playground-series-jun-2022/data.csv')
submission = pd.read_csv('../input/tabular-playground-series-jun-2022/sample_submission.csv',
                     index_col='row-col')

In [ ]:
F_4_cols = [col for col in data.columns if col.split('_')[1] == '4']

df_4 = pd.DataFrame()
for feature in data.columns:
    if feature in F_4_cols:
        df_4[feature] = data[feature]
        
df_1_2_3 = data.drop(F_4_cols, axis=1)

df_1_2_3

In [ ]:
from sklearn.impute import SimpleImputer

imputer = SimpleImputer(missing_values=np.nan,
                        strategy='mean')

imputed_1_2_3 = pd.DataFrame(imputer.fit_transform(df_1_2_3), columns=df_1_2_3.columns)

In [ ]:
imputed_1_2_3

In [ ]:
df_4

In [ ]:
%%time

from lightgbm import LGBMRegressor

for column in df_4.columns:
    train = df_4[df_4[column].isna() == False]
    
    test = df_4[df_4[column].isna() == True]
    
    X = train.drop(column, axis=1)
    y = train[column]
    
    model = LGBMRegressor(n_estimators=30000, metric='r2')
    model.fit(X, y)
    
    score = model.score(X, y)
    df_4[column][test.index] = model.predict(test.drop(column, axis=1))
    
    print(f'COLUMN: {column}, SCORE: {score}')

In [ ]:
df_4

In [ ]:
merged_1_2_3_4 = pd.concat([imputed_1_2_3, df_4], axis=1)
merged_1_2_3_4

In [ ]:
for i in tqdm(submission.index):
    row = int(i.split('-')[0])
    col = i.split('-')[1]
    submission.loc[i, 'value'] = merged_1_2_3_4.loc[row, col]
    
submission.to_csv('iterative_imputer_lgbm.csv')
submission